# FHE Aging

## Setup

In [1]:
import pandas as pd 
import pyaging as pya

In [2]:
pya.data.download_example_data('blood_chemistry_example')

|-----> 🏗️ Starting download_example_data function
|-----------> Data found in pyaging_data/blood_chemistry_example.pkl
|-----> 🎉 Done! [0.0025s]


In [3]:
df = pd.read_pickle('pyaging_data/blood_chemistry_example.pkl')

In [4]:
adata = pya.preprocess.df_to_adata(df)

|-----> 🏗️ Starting df_to_adata function
|-----> ⚙️ Create anndata object started
|-----> ✅ Create anndata object finished [0.0039s]
|-----> ⚙️ Add metadata to anndata started
|-----------? No metadata provided. Leaving adata.obs empty
|-----> ⚠️ Add metadata to anndata finished [0.0009s]
|-----> ⚙️ Log data statistics started
|-----------> There are 30 observations
|-----------> There are 10 features
|-----------> Total missing values: 0
|-----------> Percentage of missing values: 0.00%
|-----> ✅ Log data statistics finished [0.0028s]
|-----> ⚙️ Impute missing values started
|-----------> No missing values found. No imputation necessary
|-----> ✅ Impute missing values finished [0.0046s]
|-----> 🎉 Done! [0.0203s]


In [5]:
import marshal
import math
import ntpath
import os
import types
from typing import Dict, List, Tuple
from urllib.request import urlretrieve

import anndata
import numpy as np
import pandas as pd
import torch
from anndata.experimental.pytorch import AnnLoader
from torch.utils.data import DataLoader, TensorDataset

from pyaging.logger import LoggerManager, main_tqdm, silence_logger
from pyaging.models import *
from pyaging.utils import download, load_clock_metadata, progress
from pyaging.predict._postprocessing import *
from pyaging.predict._preprocessing import *

@progress("Predict ages with model")
def predict_ages_with_model(
    adata: anndata.AnnData,
    model: pyagingModel,
    device: str,
    batch_size: int,
    logger,
    indent_level: int = 2,
) -> torch.Tensor:
    """
    Predict biological ages using a trained model and input data.

    This function takes a machine learning model and input data, and returns predictions made by the model.
    It's primarily used for estimating biological ages based on various biological markers. The function
    assumes that the model is already trained. A dataloader is used because of possible memory constraints
    for large datasets.

    Parameters
    ----------
    adata : anndata.AnnData
        The AnnData object containing the dataset. Its `.X` attribute is expected to be a matrix where rows
        correspond to samples and columns correspond to features.

    model : pyagingModel
        The pyagingModel of the aging clock of interest.

    device : str
        Device to move AnnData to during inference. Eithe 'cpu' or 'cuda'.

    batch_size : int
        Batch size for the AnnLoader object to predict age.

    logger : Logger
        A logger object for logging the progress or any relevant information during the prediction process.

    indent_level : int, optional
        The indentation level for logging messages, by default 2.

    Returns
    -------
    predictions : torch.Tensor
        An array of predicted ages or biological markers, as returned by the model.

    Notes
    -----
    Ensure that the data is preprocessed (e.g., scaled, normalized) as required by the model before
    passing it to this function. The model should be in evaluation mode if it's a type that has different
    behavior during training and inference (e.g., PyTorch models).

    The exact nature of the predictions (e.g., age, biological markers) depends on the model being used.

    Examples
    --------
    >>> model = load_pretrained_model()
    >>> predictions = predict_ages_with_model(model, "cpu", logger)
    >>> print(predictions[:5])
    [34.5, 29.3, 47.8, 50.1, 42.6]

    """

    # If there is a preprocessing step
    if model.preprocess_name is not None:
        logger.info(
            f"The preprocessing method is {model.preprocess_name}",
            indent_level=indent_level + 1,
        )
    else:
        logger.info("There is no preprocessing necessary", indent_level=indent_level + 1)

    # If there is a postprocessing step
    if model.postprocess_name is not None:
        logger.info(
            f"The postprocessing method is {model.postprocess_name}",
            indent_level=indent_level + 1,
        )
    else:
        logger.info("There is no postprocessing necessary", indent_level=indent_level + 1)

    # Create an AnnLoader
    use_cuda = torch.cuda.is_available()
    dataloader = AnnLoader(adata, batch_size=batch_size, use_cuda=use_cuda)

    # with torch.no_grad():
    #     for param in model.parameters():
    #         param.zero_()

    # Use the AnnLoader for batched prediction
    predictions = []
    with torch.inference_mode():
        for batch in main_tqdm(dataloader, indent_level=indent_level + 1, logger=logger):
            batch_pred = model(batch.obsm[f"X_{model.metadata['clock_name']}"])
            predictions.append(batch_pred)
    # Concatenate all batch predictions
    predictions = torch.cat(predictions)

    return predictions

In [6]:
pya.pred.predict_age_fhe(adata, predict_ages_with_model, 'PhenoAge')

|-----> 🏗️ Starting predict_age function
|-----> ⚙️ Set PyTorch device started
|-----------> Using device: cpu
|-----> ✅ Set PyTorch device finished [0.0019s]
|-----> 🕒 Processing clock: phenoage
|-----------> ⚙️ Load clock started
|-----------------> Data found in pyaging_data/phenoage.pt
Layer: base_model.linear.weight | Size: torch.Size([1, 10]) | Values : tensor([[-0.0336,  0.0095,  0.1953,  0.0954, -0.0120,  0.0268,  0.3306,  0.0019,
          0.0554,  0.0804]], dtype=torch.float64, grad_fn=<SliceBackward0>) 

Layer: base_model.linear.bias | Size: torch.Size([1]) | Values : tensor([-19.9067], dtype=torch.float64, grad_fn=<SliceBackward0>) 

|-----------> ✅ Load clock finished [0.0453s]
|-----------> ⚙️ Check features in adata started
|-----------------> All features are present in adata.var_names.
|-----------> ✅ Check features in adata finished [0.0037s]
|-----------> ⚙️ Predict ages with model started
|-----------------> There is no preprocessing necessary
|-----------------> Th

In [7]:
# pya.pred.predict_age(adata, predict_ages_with_model, 'PhenoAge')

In [8]:
adata.obs.head()

,phenoage
patient1,74.348798
patient2,67.372000
patient3,74.789739
patient4,46.991769
patient5,44.559486


In [9]:
adata

AnnData object with n_obs × n_vars = 30 × 10
    obs: 'phenoage'
    var: 'percent_na'
    uns: 'phenoage_percent_na', 'phenoage_missing_features', 'phenoage_metadata'
    layers: 'X_original'

In [10]:
combined_df = pd.DataFrame({
    'phenoage': adata.obs["phenoage"],
    'chronological_age': df["age"]
})

# Display the first few rows
combined_df[:15]

,phenoage,chronological_age
patient1,74.348798,70.2
patient2,67.372000,76.5
patient3,74.789739,66.4
patient4,46.991769,46.5
patient5,44.559486,42.3
patient6,72.509460,76.9
patient7,57.377050,55.1
patient8,31.779798,34.6
patient9,50.356509,47.3
patient10,67.696706,52.3


## Manual Torch Execution

In [11]:
class PhenoAge(pyagingModel):
    def __init__(self):
        super().__init__()

    def preprocess(self, x):
        return x

    def postprocess(self, x):
        """
        Applies a convertion from a CDF of the mortality score from a Gompertz
        distribution to phenotypic age.
        """
        # lambda
        l = torch.tensor(0.0192, device=x.device, dtype=x.dtype)
        mortality_score = 1 - torch.exp(-torch.exp(x) * (torch.exp(120 * l) - 1) / l)
        age = 141.50225 + torch.log(-0.00553 * torch.log(1 - mortality_score)) / 0.090165
        return age

In [12]:
outer_model = PhenoAge()

In [13]:
device = "cpu"
weights_path = f"./pyaging_data/phenoage.pt"
clock = torch.load(weights_path, weights_only=False)

for name, param in clock.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

# Prepare clock for inference
clock.to(torch.float64)
clock.to(device)
clock.eval()

Layer: base_model.linear.weight | Size: torch.Size([1, 10]) | Values : tensor([[-0.0336,  0.0095,  0.1953,  0.0954, -0.0120,  0.0268,  0.3306,  0.0019,
          0.0554,  0.0804]], dtype=torch.float64, grad_fn=<SliceBackward0>) 

Layer: base_model.linear.bias | Size: torch.Size([1]) | Values : tensor([-19.9067], dtype=torch.float64, grad_fn=<SliceBackward0>) 



PhenoAge(
  (base_model): LinearModel(
    (linear): Linear(in_features=10, out_features=1, bias=True)
  )
)

In [14]:
adata

AnnData object with n_obs × n_vars = 30 × 10
    obs: 'phenoage'
    var: 'percent_na'
    uns: 'phenoage_percent_na', 'phenoage_missing_features', 'phenoage_metadata'
    layers: 'X_original'

In [15]:
clock.metadata['clock_name']

'phenoage'

In [16]:
# dataloader = AnnLoader(adata, batch_size=1024, use_cuda=False)

# # Use the AnnLoader for batched prediction
# predictions = []
# with torch.inference_mode():
#     for batch in main_tqdm(dataloader):
#         print(batch.obsm)
#         batch_pred = clock(batch.obsm[f"X_{clock.metadata['clock_name']}"])
#         predictions.append(batch_pred)
# # Concatenate all batch predictions
# predictions = torch.cat(predictions)

In [17]:
adata.obs

,phenoage
patient1,74.348798
patient2,67.372000
patient3,74.789739
patient4,46.991769
patient5,44.559486
patient6,72.509460
patient7,57.377050
patient8,31.779798
patient9,50.356509
patient10,67.696706


In [32]:
dataset = df.iloc[:, 0:10]

In [22]:
with torch.inference_mode():
    adata

|-----> in progress: 100.0000%
ello: AnnCollectionView object with n_obs × n_vars = 30 × 10
    layers: 'X_original'
    obs: 'phenoage' <class 'anndata.experimental.multi_files._anncollection.AnnCollectionView'>


In [27]:
df.columns

Index(['albumin', 'creatinine', 'glucose', 'log_crp', 'lymphocyte_percent',
       'mean_cell_volume', 'red_cell_distribution_width',
       'alkaline_phosphatase', 'white_blood_cell_count', 'age'],
      dtype='object')